In [ ]:
from typing import Any, Mapping, Sequence, cast
from pathlib import Path
import json
import shutil
import sys
import importlib.metadata as im

from phaseedge.jobs.store_ce_model import lookup_ce_by_key
from phaseedge.schemas.calc_spec import CalcSpec
from phaseedge.storage.cetrainref_dataset import Dataset
from phaseedge.storage.store import lookup_total_energy_eV


def _strip_kfs3_prefix(dir_name: str, anchor: str = "/kfs3/scratch/cbu/") -> Path:
    """
    Strip everything before the /kfs3/scratch/cbu/ anchor from an energy_result.dir_name.

    Example
    -------
    x3107c0s13b0n0.head.cm.kestrel.hpc.nrel.gov:/kfs3/scratch/cbu/block_.../launcher_...
    -> /kfs3/scratch/cbu/block_.../launcher_...

    Parameters
    ----------
    dir_name : str
        Raw dir_name from the energy_result.
    anchor : str
        Anchor substring that marks the beginning of the filesystem path.

    Returns
    -------
    Path
        Path starting at /kfs3/scratch/cbu/...
    """
    idx = dir_name.find(anchor)
    if idx == -1:
        raise RuntimeError(
            f"dir_name does not contain expected anchor {anchor!r}: {dir_name!r}"
        )
    stripped = dir_name[idx:]
    return Path(stripped)


def organize_results(ce_key: str, name: str, ordered_task_id: str) -> None:
    """
    For a given CE key, validate that all associated energy_results are converged and
    have existing directories on disk, then organize them into a new directory tree.

    Steps
    -----
    1. Load CE doc and dataset.
    2. For each train_ref:
       - Fetch energy_result.
       - Require that max_force_eV_per_A <= calc_spec.max_force_eV_per_A.
       - Strip dir_name to /kfs3/scratch/cbu/... and require that directory exists.
       - Collect (occ_key, stripped_dir_path, train_ref).
       If *any* check fails, raise an error (fail loudly).
    3. Create /scratch/cbu/mp/{name}/.
    4. For each (occ_key, stripped_dir_path, ref):
       - Create /scratch/cbu/mp/{name}/{occ_key}/.
       - Copy the contents of stripped_dir_path into that directory.
       - Write a JSON file with the 6 fields:
         ordered_task_id, prototype, prototype_params, composition_map,
         configurational_structure, supercell_diag.

    Parameters
    ----------
    ce_key : str
        Cluster-expansion key whose training-set results should be organized.
    name : str
        Name of the new folder under /scratch/cbu/mp/, e.g. "test" -> /scratch/cbu/mp/test/.
    ordered_task_id : str
        The durable ordered parent task id to record in each config JSON.
    """
    # ---- Load CE document ----
    doc = lookup_ce_by_key(ce_key)
    if not doc:
        raise RuntimeError(f"No CE found for ce_key={ce_key}")

    calc_spec_top = CalcSpec.from_dict(cast(Mapping[str, Any], doc["calc_spec"]))
    dataset_key = cast(str, doc["dataset_key"])
    dataset = Dataset.from_key(dataset_key)

    # supercell_diag: stored in the CE doc; ensure it's a 3-tuple of ints
    sx, sy, sz = (int(x) for x in doc["supercell_diag"])
    supercell_diag = (sx, sy, sz)

    # prototype + params: pull from prototype_spec dict in the CE doc
    prototype_spec_data = cast(Mapping[str, Any], doc.get("prototype_spec", {}))
    prototype = str(prototype_spec_data.get("prototype", ""))

    # Try to extract numeric prototype params from a nested "params" dict if present;
    # otherwise fall back to an empty dict. You can tighten this once you settle the schema.
    raw_params = prototype_spec_data.get("params", {})  # type: ignore[assignment]
    if isinstance(raw_params, Mapping):
        prototype_params: dict[str, float] = {}
        for k, v in raw_params.items():
            try:
                prototype_params[str(k)] = float(v)
            except (TypeError, ValueError):
                # Skip non-numeric params; adjust if you want to preserve them differently.
                continue
    else:
        prototype_params = {}
    if 'spinel' in prototype and 'u' not in prototype_params:
        # Special case: add default u parameter for spinels if missing
        prototype_params['u'] = 0.36

    train_refs: Sequence[Any] = dataset.train_refs

    print(f"CE key      : {ce_key}")
    print(f"dataset_key : {dataset_key}")
    print(f"n train_refs: {len(train_refs)}")
    print(f"supercell_diag: {supercell_diag}")
    print(f"prototype      : {prototype}")
    print(f"prototype_params (numeric only): {prototype_params}")

    # ---- First pass: validate everything and collect paths + refs ----
    collected: list[tuple[str, Path, Any]] = []

    for i, ref in enumerate(train_refs):
        composition_map = ref.composition_map
        occ_key = ref.occ_key
        calc_spec = ref.calc_spec

        # Sanity: warn if calc_spec differs from CE-level spec, but still use ref.calc_spec
        if calc_spec != calc_spec_top:
            print(
                f"[{i:4d}] WARNING: calc_spec mismatch between CE doc and train_ref; "
                "using ref.calc_spec for lookup."
            )

        energy_result = lookup_total_energy_eV(
            occ_key=occ_key,
            calc_spec=calc_spec,
        )
        if energy_result is None:
            raise RuntimeError(
                f"[{i}] Missing energy_result for occ_key={occ_key}, "
                f"composition_map={composition_map}"
            )

        # Convergence check
        if energy_result.max_force_eV_per_A > calc_spec.max_force_eV_per_A:
            raise RuntimeError(
                f"[{i}] Unconverged energy_result for occ_key={occ_key}, "
                f"composition_map={composition_map}. "
                f"max|F|={energy_result.max_force_eV_per_A:.3f} eV/Å "
                f"> threshold={calc_spec.max_force_eV_per_A:.3f} eV/Å"
            )

        # Strip the host prefix to get /kfs3/scratch/cbu/...
        raw_dir_name = energy_result.dir_name
        stripped_path = _strip_kfs3_prefix(raw_dir_name)

        # Require that the directory exists
        if not stripped_path.is_dir():
            raise RuntimeError(
                f"[{i}] Directory does not exist on disk: {stripped_path} "
                f"(occ_key={occ_key}, composition_map={composition_map})"
            )

        collected.append((occ_key, stripped_path, ref))

    print("All energy_results are converged and all directories exist. Proceeding to copy.")

    # ---- Prepare target root directory under /scratch/cbu/mp/{name} ----
    target_root = Path("/scratch/cbu/mp") / name
    if target_root.exists():
        raise RuntimeError(
            f"Target directory already exists: {target_root}. "
            "Refusing to overwrite; please choose a different name or clean up manually."
        )

    target_root.mkdir(parents=True, exist_ok=False)
    print(f"Created target root: {target_root}")

    # ---- Second pass: copy into /scratch/cbu/mp/{name}/{occ_key}/ and write JSON ----
    for i, (occ_key, src_dir, ref) in enumerate(collected):
        dest_dir = target_root / occ_key

        if dest_dir.exists():
            raise RuntimeError(
                f"[{i}] Destination directory already exists: {dest_dir}. "
                "This should not happen; aborting to avoid clobbering."
            )

        print(f"[{i:4d}] Copying {src_dir} -> {dest_dir}")
        shutil.copytree(src_dir, dest_dir, dirs_exist_ok=False)

        # Build the 6-field JSON payload for this configuration
        composition_map = ref.composition_map
        structure = ref.structure  # configurational structure used for CE, not relaxed DFT geom

        payload = {
            # 1) ordered_task_id: durable parent ordered doc id
            "ordered_task_id": ordered_task_id,
            # 2) prototype: prototype tag (string)
            "prototype": prototype,
            # 3) prototype_params: numeric prototype parameters
            "prototype_params": prototype_params,
            # 4) composition_map: per-sublattice counts
            "composition_map": composition_map,
            # 5) reference_structure: structure used in CE (as a dict for JSON)
            "reference_structure": structure.as_dict(),
            # 6) supercell_diag: stored as a simple list for JSON
            "supercell_diag": list(supercell_diag),
            "versions": {
                "python_version": sys.version.split()[0],
                "smol_version": im.version("smol"),
                "phase_edge_version": "1.0.1",
                "pymatgen_version": im.version("pymatgen"),
                "ase_version": im.version("ase"), 
                "numpy_version": im.version("numpy"),
                "pre_relax_model": "TensorNet-MatPES-r2SCAN-v2025.1-PES",
            }
        }

        json_path = dest_dir / "disordered_task_doc_metadata.json"
        with json_path.open("w", encoding="utf-8") as fh:
            json.dump(payload, fh, indent=2, sort_keys=True)
        print(f"[{i:4d}] Wrote metadata JSON: {json_path}")

    print("Done organizing results.")
    print(f"All data copied under: {target_root}")


organize_results("25ae33c4abb83c7fffa4dc76b972b11b2fec3756b9bf72ab56edd4e70dc4013a", "MgAl2O4_02eVpA_mp-3536", ordered_task_id="mp-2029216")


CE key      : 25ae33c4abb83c7fffa4dc76b972b11b2fec3756b9bf72ab56edd4e70dc4013a
dataset_key : 2bbddab64a48cd779b1d29d535b4681a9ea2935217fd2d8fd81911565f48693f
n train_refs: 500
supercell_diag: (2, 2, 2)
prototype      : spinel_Q0O
prototype_params (numeric only): {'a': 8.08, 'u': 0.36}
All energy_results are converged and all directories exist. Proceeding to copy.
Created target root: /scratch/cbu/mp/MgAl2O4_02eVpA
[   0] Copying /kfs3/scratch/cbu/block_2025-11-23-18-02-00-998017/launcher_2025-11-24-06-24-37-110168/launcher_2025-11-24-14-50-43-435875 -> /scratch/cbu/mp/MgAl2O4_02eVpA/dab1888592b5999bedc310b5eea4346c06f006a6087c86b66bcfe72b1ac52020
[   0] Wrote metadata JSON: /scratch/cbu/mp/MgAl2O4_02eVpA/dab1888592b5999bedc310b5eea4346c06f006a6087c86b66bcfe72b1ac52020/disordered_config_task.json
[   1] Copying /kfs3/scratch/cbu/block_2025-11-23-18-02-00-998017/launcher_2025-11-24-06-13-12-921602/launcher_2025-11-24-14-50-50-026677 -> /scratch/cbu/mp/MgAl2O4_02eVpA/d99a739078c9e1730effe

In [1]:
from pymatgen.io.validation import VaspValidator
validation_doc = VaspValidator.from_directory("/kfs3/scratch/cbu/block_2025-12-05-22-14-08-263137/launcher_2025-12-06-14-14-18-251409/launcher_2025-12-06-14-56-07-202352", fast=False)
validation_doc.reasons

['INPUT SETTINGS --> GGA_COMPAT: is False, but should be == True.',
 'INPUT SETTINGS --> MAX FINAL FORCE MAGNITUDE: is 0.041, but should be <= 0.02.']